In [9]:
import importlib
from dbm import error
import ast
from optparse import Option
from time import process_time_ns
import traceback
import time
import pandas as pd
import numpy as np
import warnings
import plotly.graph_objects as go
from pandas import date_range, notna
from pandas.errors import SettingWithCopyWarning
from plotly.subplots import make_subplots
import functions_straddle as fn
import process as dp
import plotly.io as pio
import psycopg2
conn = psycopg2.connect(dbname="qdap_test",user="amt",password="amt",host="192.168.2.23",port="5432"  )
cursor = conn.cursor()
importlib.reload(dp); importlib.reload(fn)
warnings.simplefilter(action='ignore', category=FutureWarning);warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
dt = 'date_timestamp'

In [10]:
importlib.reload(dp); importlib.reload(fn)
all_trades, all_execs , trade_id = {} , {} ,0
ranges = pd.date_range(start='2024-08-01', end='2024-08-30')

all_running_pnl, all_call_movement, all_put_movement, all_call_iv, all_put_iv = [],[],[],[],[]
list_symbol = ['BANKNIFTY']
for symbol in list_symbol:
    all_trades[symbol],all_execs[symbol] = [] , []
    for d_t in ranges:
        # try:
            futures = dp.FetchFuturesByDate(cursor,symbol,d_t)
            if not futures.empty:
                options = dp.FetchOptionsByDateCsv(symbol,d_t.date())
                print(symbol,str(d_t.date()))
                gen_call, gen_put = options[options['opt_type'] == 'CE'], options[options['opt_type'] == 'PE']
                # call, put = dp.fetch_call_put_(options)
                # call.ffill(inplace=True) ; put.ffill(inplace=True)
                eh=9; em = 16
                trade_id,execs, trades, running_pnl, difference, delta_put, delta_call, iv, ivs = fn.generate_trades_syn(trade_id,eh,em,futures, gen_call, gen_put, io=0, buying='open', selling='close', DeltaLB=0.2,delta_range=0.1, DeltaUB = 0.55,initial_capital=10000)
                dp.plotter([running_pnl,iv,ivs[4]],['pnl','iv','1','2','3','4','5'],filename=f'{d_t.date()}.html')
                all_running_pnl.extend(running_pnl)
                all_trades[symbol].extend(trades)
                all_execs[symbol].extend(execs)
                if len(trades) == 0:
                    print(f"No trades took place on {d_t}")
        # except Exception as e:
            # Print the error message
            # Print the traceback for more details
            


BANKNIFTY 2024-08-01
BANKNIFTY 2024-08-02
BANKNIFTY 2024-08-05
BANKNIFTY 2024-08-06
BANKNIFTY 2024-08-07
BANKNIFTY 2024-08-08
BANKNIFTY 2024-08-09
BANKNIFTY 2024-08-12
BANKNIFTY 2024-08-13
BANKNIFTY 2024-08-14
BANKNIFTY 2024-08-16
BANKNIFTY 2024-08-19
BANKNIFTY 2024-08-20
BANKNIFTY 2024-08-21
BANKNIFTY 2024-08-22
BANKNIFTY 2024-08-23
BANKNIFTY 2024-08-26
BANKNIFTY 2024-08-27
BANKNIFTY 2024-08-28


ValueError: Input y contains NaN.

In [51]:
print(running_pnl)

[np.int64(0), (Timestamp('2020-01-01 09:16:00'), np.float64(1.4792)), np.int64(4), (Timestamp('2020-01-01 09:17:00'), np.float64(1.4792)), np.int64(7), (Timestamp('2020-01-01 09:18:00'), np.float64(1.4792)), np.int64(6), (Timestamp('2020-01-01 09:19:00'), np.float64(1.4792)), np.int64(8), (Timestamp('2020-01-01 09:20:00'), np.float64(1.4792)), np.int64(9), (Timestamp('2020-01-01 09:21:00'), np.float64(1.4792)), np.int64(7), (Timestamp('2020-01-01 09:22:00'), np.float64(1.4792)), np.int64(4), (Timestamp('2020-01-01 09:23:00'), np.float64(1.4792)), np.int64(3), (Timestamp('2020-01-01 09:24:00'), np.float64(1.4792)), np.int64(0), (Timestamp('2020-01-01 09:25:00'), np.float64(1.4792)), np.int64(-1), (Timestamp('2020-01-01 09:26:00'), np.float64(1.4792)), np.int64(-1), (Timestamp('2020-01-01 09:27:00'), np.float64(1.4792)), np.int64(0), (Timestamp('2020-01-01 09:28:00'), np.float64(1.4792)), np.int64(-2), (Timestamp('2020-01-01 09:29:00'), np.float64(1.4792)), np.int64(-2), (Timestamp('2020

In [29]:
streams = [all_running_pnl,all_call_movement,all_put_movement,all_call_iv,all_put_iv]
labels = ['running pnl','call_strike','put_strike','civ','piv']
dp.plotter(streams,labels)

IndexError: list index out of range

In [31]:
opt = dp.fetch_options_('options_BANKNIFTY_2020-01-03')
strik = 0
x = opt[(opt['strike']==strik)]
dp.plotter([[x]],["NAE"])

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\tanmay\\Documents\\db\\o_g\\options_BANKNIFTY_2020-01-03_G.csv'

In [312]:

importlib.reload(dp); importlib.reload(fn)
columns = ['entry_time','entry_price','exit_price','exit_time','type','pnl','strike','entry_delta','exit_delta','sell_id','buy_id']
fn.to_excl(all_trades['NIFTY'],columns,'All Trades')

In [32]:
pri_ = {'id':1,'time': 1,'opt_type':1,'log':1,'delta':1,'strike': 1, 'price': 1,'pnl': 0, 'trade_type': 1,'transaction_cost': 0, 'slippage': 0,}
fn.print_execs_to_html(all_execs['NIFTY'],pri_,filename=f'all exec log.html')

columns_to_print = {'entry_time':1 , 'entry_price':1 ,'exit_price':1, 'exit_time' : 1 ,  'time':1 , 'movement': 0, 'type':1 , 'pnl':1,'strike':1,'entry_delta':1,'exit_delta':1,'sell_id':1,'buy_id':1}
fn.print_trades_to_html(all_trades['NIFTY'], columns_to_print , filename=f'all trade logs.html')

sharpie_1, stats_1 = fn.sharpie_straddle(all_trades['NIFTY'])
print(sharpie_1)


KeyError: 'NIFTY'

In [325]:
pri_ = {'id':1,'time': 1,'opt_type':1,'log':1,'delta':1,'strike': 1, 'price': 1,'pnl': 0, 'trade_type': 1,'transaction_cost': 0, 'slippage': 0,}
fn.print_execs_to_html(all_execs['BANKNIFTY'],pri_,filename=f'all exec log.html')

columns_to_print = {'entry_time':1 , 'entry_price':1 ,'exit_price':1, 'exit_time' : 1 ,  'time':1 , 'movement': 0, 'type':1 , 'pnl':1,'strike':1,'entry_delta':1,'exit_delta':1,'sell_id':1,'buy_id':1}
fn.print_trades_to_html(all_trades['BANKNIFTY'], columns_to_print , filename=f'all trade logs.html')

sharpie, stats = fn.sharpie_straddle(all_trades['BANKNIFTY'])
print(sharpie)


C:\Users\Tanmay\Desktop\Files\all exec log.html

C:\Users\Tanmay\Desktop\Files\all trade logs.html

0.11968604464498551
